In [0]:
#Priya Rajpurohit 2015073
#Sakshi Saini 2017092

In [0]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

Using TensorFlow backend.


In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers.core import Layer
import keras.backend as K

from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

import cv2 
import numpy as np 
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils

import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split


In [0]:
# ds = tfds.load("plant_village", split=tfds.Split.TRAIN, batch_size=-1)
# ds = tfds.as_numpy(ds)

# images, labels = ds["image"], ds["label"]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -uq "/content/drive/My Drive/SML_Project/Segmented.zip" 

In [0]:
import os

path = '/content/Segmented/'
images=[]
labels=[]
# r=root, d=directories, f = files
lable = 1
labels = []
for r, d, f in os.walk(path):
    for folder in d:
        
        for r1, d1, f1 in os.walk(os.path.join(r, folder)):
          for file in f1:
            # print(file)
            images.append( cv2.imread(os.path.join(r1, file) ))
            labels.append(lable)

          lable = lable +1
images=np.array(images)
images.shape

(48153, 256, 256, 3)

In [0]:
x_train, x_test, y_train, y_test = train_test_split( images[:20000], labels[:20000], test_size=0.2, random_state=42, shuffle=True )
print(x_train.shape, x_test.shape)


(16000, 256, 256, 3) (4000, 256, 256, 3)


In [0]:
from keras.models import Model

image_input = Input(shape=(224, 224, 3))

res = ResNet50(input_tensor=image_input, weights='imagenet', include_top=False)

for layer in res.layers:
  layer.trainable = False
x = GlobalAveragePooling2D()(res.output)
x=Dropout(0.3)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x)


prediction = Dense(38, activation='softmax')(x)

# create a model object
model = Model(inputs=res.input, outputs=prediction)
model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________

In [0]:
# # X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
# # X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:,:]])

# resized = []
# n = 224
# m = 224

# for image in images:
#   image = cv2.resize( image, (n, m) )
#   resized.append(image)

#images, labels = ds["image"], ds["label"]
n1 = 224
m1 = 224

features = []
for image in x_train:

  image = cv2.resize(image , (n1,m1) )

  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  feature = model.predict(image)
  
  features.append(feature)

featurestest = []
for image in x_test:

  image = cv2.resize(image , (n1,m1) )

  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  feature = model.predict(image)
  
  featurestest.append(feature)

In [0]:
features = np.array(features)
featurestest=np.array(featurestest)



In [0]:
print(featurestest.shape)
print(features.shape)

(4000, 1, 38)
(16000, 1, 38)


In [0]:
from sklearn.ensemble import RandomForestClassifier

features = features.reshape(16000,38)

clf = RandomForestClassifier()
clf.fit(features,  y_train)
featurestest = featurestest.reshape(4000, 38)

accuracy = clf.score(featurestest,y_test)
accuracy *= 100
print(accuracy)
import h5py


h5_data    = 'segmented_resnet.h5'
h5_labels  = 'labels_resnet_seg.h5'

h5f_data = h5py.File(h5_data, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(h5_labels, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(labels))

import pickle


filename = 'resnet_rf_model_color.sav'
pickle.dump(clf, open(filename, 'wb'))

82.675


In [0]:
from sklearn import svm

features = features.reshape(16000,38)

clf1 = svm.SVC()
clf1.fit(features,  y_train)
featurestest = featurestest.reshape(4000, 38)

accuracy = clf1.score(featurestest,y_test)
accuracy *= 100
print(accuracy)
import h5py





filename = 'resnet_svm_model_color.sav'
pickle.dump(clf1, open(filename, 'wb'))

84.675


In [0]:
!pip3 install elm
!pip3 install --upgrade numpy folium imgaug

     |████████████████████████████████| 20.2MB 253kB/s 
     |████████████████████████████████| 92kB 12.4MB/s 
     |████████████████████████████████| 952kB 61.2MB/s 
ERROR: elm 0.1.3 has requirement numpy==1.15.4, but you'll have numpy 1.18.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.10.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.4.0 which is incompatible.
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [0]:
import elm
elmk=elm.ELMKernel()
elmk.search_param(data, cv="kfold", of="accuracy", eval=10)

tr_result = elmk.train(features,y_train)
te_result = elmk.test(featurestest,y_test)

print(te_result.get_accuracy)


ContextualVersionConflict: ignored